In [192]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, log_loss

In [307]:
# To get basic data of a team in a tournament
# def getTeamDataByYear(year, team):
#     basic_data = pd.read_excel(io="./data/formatted/basic/basicdata" + year + ".xlsx")
#     row = basic_data.loc[basic_data['School'] == team]
#     team_data = row.copy() #prevent modifying basic_data
#     team_data.drop('ID', axis=1, inplace=True)
#     team_data.drop('School', axis=1, inplace=True)
#     team_data['teamSeed'] = getTeamSeedByYear(year, team)
#     return team_data
    
# def getTeamSeedByYear(year, team):
#     games = pd.read_csv("./data/playoff/playoffdata" + year + ".csv")
#     first = breakdownBracket(games)[0]
#     for index, row in first.iterrows():
#         if (row['winnerName'] == team):
#             return row['winnerSeed']
#         elif (row['loserName'] == team):
#             return row['loserSeed']
#     print("cannot find team %s seed", team)
#     return -1

# def calculateLabelsPerYear(year, games):
#     playoff_results = pd.read_csv("./data/playoff/playoffdata" + year + ".csv")
#     winner_names = playoff_results['winnerName']
#     labels = winner_names.copy()
#     for index, row in games.iterrows():
#         if row['teamAName'] == winner_names.loc[index]:
#             labels.loc[index] = 1
#         else:
#             labels.loc[index] = 0
#     return labels.to_frame()
    
 
# find difference between two teams' stats
# def compareTwoTeams(teamAName, teamBName, year):
#     team_A = getTeamDataByYear(year, teamAName)
#     team_B = getTeamDataByYear(year, teamBName)
#     if len(team_A.columns) == len(team_B.columns):
#         diff = team_A.copy()
#         for col in team_A.columns:
#             diff[col] = team_A[col].values[0] - team_B[col].values[0]
#         return diff
#     else: 
#         print("Team A column length: ", len(team_A.columns))
#         print("Team B column length: ", len(team_B.columns))
#         print("Something is wrong in the data")
#         return None

# def extractData(data, year):
#     x_train = None
#     for index, game in data.iterrows():
#         diff = compareTwoTeams(game['teamAName'], game['teamBName'], year)
#         x_train = diff if x_train is None else pd.concat([x_train, diff])
#     return x_train


In [176]:
# ## Train and Test with one year
# def getTestingData(target_year):
#     train_data = sortAllGamesBySeed(target_year)
#     teams_first, second, third, fourth, semi, final = breakdownBracket(train_data)

#     train_labels = calculateLabelsPerYear(target_year, train_data)
#     labels_first, y_second, y_third, y_fourth, y_semi, y_final = breakdownBracket(train_labels)
    
#     x_test = extractData(teams_first, target_year)
#     x_test.drop('PersonalFouls', axis=1, inplace=True)
#     return x_test, labels_first

In [77]:
y_test = y_second['winnerName'].values.tolist()
accuracy.append(np.mean(preds == y_test))
# print ("Finished iteration:", i)
print ("The accuracy is", sum(accuracy)/len(accuracy))

The accuracy is 0.875


In [362]:
#### DIFFERENT FROM ABOVE
#### USE ALL DATA TO TRAIN

# private method used in sortAllGamesBySeed
def swap(row):   
    d = {
        'season': [row.pop('season')],
        'teamAName': [row.pop('teamBName')],
        'teamASeed': [row.pop('teamBSeed')],
        'teamBName': [row.pop('teamAName')],
        'teamBSeed': [row.pop('teamASeed')],
    }
    updated_row = pd.DataFrame(data=d)
    return updated_row
    
# left = higherSeed, right = lowerSeed
def sortAllGamesBySeed(year):
    playoff_data = pd.read_csv("./data/playoff/playoffdata" + year + ".csv")

    ## Swap columns so higher seed (lower value) on the left, lower seed (higher value) on the right
    updated_data = playoff_data.copy()
    updated_data.columns = ['season', 'teamAName', 'teamASeed', 'teamBName', 'teamBSeed']
    for index, row in updated_data.iterrows():
        if row['teamASeed'] > row['teamBSeed']:
            updated_data.loc[index,['teamAName', 'teamBName']] = updated_data.loc[index,['teamBName', 'teamAName']].values
            updated_data.loc[index,['teamASeed', 'teamBSeed']] = updated_data.loc[index,['teamBSeed', 'teamASeed']].values
    return updated_data


# break down the bracket into rounds
def breakdownBracket(updated_data):
    first_round = None
    second_round = None
    third_round = None
    fourth_round = None
    semi_final = updated_data.iloc[60 : 62]
    final = updated_data.iloc[62 : 63]
    for region_index in range(0, 60, 15): 
        first = updated_data.iloc[region_index : region_index + 8]
        first_round = first if first_round is None else pd.concat([first_round, first])

        second = updated_data.iloc[region_index + 8 : region_index + 12, :]  
        second_round = second if second_round is None else pd.concat([second_round, second])

        third = updated_data.iloc[region_index + 12 : region_index + 14]
        third_round = third if third_round is None else pd.concat([third_round, third])

        fourth = updated_data.iloc[region_index + 14 : region_index + 15]
        fourth_round = fourth if fourth_round is None else pd.concat([fourth_round, fourth])
    return first_round, second_round, third_round, fourth_round, semi_final, final


def swapGames(playoff_data):
#     playoff_data = pd.read_csv("./data/playoff/playoffdata" + year + ".csv")

    ## Swap columns so higher seed (lower value) on the left, lower seed (higher value) on the right
    updated_data = playoff_data.copy()
    updated_data.columns = ['season', 'teamAName', 'teamASeed', 'teamBName', 'teamBSeed']
    for index, row in updated_data.iterrows():
        if row['teamASeed'] > row['teamBSeed']:
            updated_data.loc[index,['teamAName', 'teamBName']] = updated_data.loc[index,['teamBName', 'teamAName']].values
            updated_data.loc[index,['teamASeed', 'teamBSeed']] = updated_data.loc[index,['teamBSeed', 'teamASeed']].values
    return updated_data

def calculateLabels(year, games):
    playoff_results = pd.read_csv("./data/playoff/playoffdata" + year + ".csv")
    winner_names = playoff_results['winnerName']
    labels = winner_names.copy()
    for index, row in games.iterrows():
        if row['teamAName'] == winner_names.loc[index]:
            labels.loc[index] = 1
        else:
            labels.loc[index] = 0
    return labels.to_frame()


def compareTeams(team_a_info, team_b_info):
    if len(team_a_info.columns) == len(team_b_info.columns):
        diff = team_a_info.copy()
        for col in team_a_info.columns:
            diff[col] = team_a_info[col].values - team_b_info[col].values
        return diff
    else: 
        print("Team A column length: ", len(team_A.columns))
        print("Team B column length: ", len(team_B.columns))
        print("Something is wrong in the data")
        return None


def getAllGames(years):
    all_games = None
    all_labels = None
    for year in years:
        data = pd.read_csv("./data/playoff/playoffdata" + year + ".csv")
        playoff_data = swapGames(data.copy())
        labels = calculateLabels(year, playoff_data)
    
        data = pd.read_excel(io="./data/formatted/basic/basicdata" + year + ".xlsx")
        team_basic_data = data.copy()
        
        team_a = playoff_data.iloc[:, 1:3]
        team_b = playoff_data.iloc[:, 3:5]
        
        team_a = team_a.merge(team_basic_data, left_on='teamAName', right_on='School', how='left')
        team_a = team_a.rename(columns={"teamASeed": "seed"})
        team_a.drop('teamAName', axis=1, inplace=True)
#         team_a.drop('teamASeed', axis=1, inplace=True)
        team_a.drop('School', axis=1, inplace=True)
        team_a.drop('ID', axis=1, inplace=True)
        team_a.drop('PersonalFouls', axis=1, inplace=True)
        
        team_b = team_b.merge(team_basic_data, left_on='teamBName', right_on='School', how='left')
        team_b = team_b.rename(columns={"teamBSeed": "seed"})
        team_b.drop('teamBName', axis=1, inplace=True)
#         team_b.drop('teamBSeed', axis=1, inplace=True)
        team_b.drop('School', axis=1, inplace=True)
        team_b.drop('ID', axis=1, inplace=True)
        team_b.drop('PersonalFouls', axis=1, inplace=True)
        
        diff_games = compareTeams(team_a, team_b)
        
        all_labels = labels if all_labels is None else pd.concat([all_labels, labels])
        all_games = diff_games if all_games is None else pd.concat([all_games, diff_games])
    
    return all_games, all_labels


def normalizeData(data): 
    updated_data = data.copy()
    for col in data.columns:
        rows = data[col].values
        min_val = min(rows)
        rows = (rows + abs(min_val))
        updated_data.update(pd.DataFrame({col: rows}))
    return updated_data

test_years = [
    '201617', '201718','201819'
]
train_years = [
    '200001','200102', '200203', '200304', '200405', '200506', '200607', '200708', '200809', '200910',
    '201011', '201112', '201213','201314', '201415','201516'
]
selected = 1

In [364]:
x_train, y_train = getAllGames(train_years)
x_test, y_test = getAllGames([test_years[selected]])

In [365]:
# norm_x_train = normalizeData(x_train)
# norm_x_test = normalizeData(x_test)
# print(norm_x_train)

In [378]:
gbr = GradientBoostingRegressor(n_estimators=100, max_depth=5)
# rfc = RandomForestClassifier(n_estimators=64)

In [375]:
def execute(model, x_train, y_train, x_test, y_test, epochs):
    for i in range(epochs):
        model.fit(x_train, y_train['winnerName'].tolist())
    pred = model.predict(x_test)
#     pred_proba = model.predict_proba(x_test)
    mse = mean_squared_error(y_test, pred)
    print("MSE: ", mse)
#     print("pred:", pred)
#     print("pred_proba:", pred_proba)
#     print("Log Loss for Proba: ", log_loss(y_test, pred_proba))
    print("Score:", model.score(x_test, y_test))
    print('----------------------------------------')
    return pred

def findWinners(predicted_labels, games):
    team_a_names = games['teamAName'].values
    team_b_names = games['teamBName'].values
    winners = list()
    for i in range(len(predicted_labels)):
        winner = predicted_labels[i]
        if winner == 1: 
            winners.append(team_a_names[i])
        elif winner == 0: 
            winners.append(team_b_names[i])
    return winners

In [379]:
pred_labels = execute(rfc, x_train, y_train, x_test, y_test, 100)


MSE:  0.10975130053550407
Score: 0.49348498624951687
----------------------------------------


In [ ]:
MSE:  0.19047619047619047
Log Loss for Proba:  0.4015518423510818
Score: 0.8095238095238095
----------------------------------------
MSE:  0.1746031746031746
Log Loss for Proba:  0.40287693665755603
Score: 0.8253968253968254

In [238]:
# team_names, expected_winner = getTestingData('201819')
# result = model.predict(team_names)
# result_mse = mean_squared_error(expected_winner, result)
# print("MSE:", result_mse)
# print("Log Loss:", log_loss(expected_winner, result))
# accurracy(result)

MSE: 0.2917077840735064
Log Loss: 0.9039714613295866
accurracy: 0.84375


In [386]:
data = pd.read_csv("./data/playoff/playoffdata" + test_years[selected] + ".csv")
playoff_data = swapGames(data.copy())
games = breakdownBracket(playoff_data)[0]
winners = pd.DataFrame({'winnerName': data['winnerName'].tolist(), 'seed': data['winnerSeed'].tolist()})
actual_winners = breakdownBracket(winners)[0]['winnerName'].tolist()
actual_seeds = breakdownBracket(winners)[0]['seed'].tolist()
winner_names = findWinners(pred_labels[:len(games)], games)

[1.01329661 0.01589287 0.98492134 0.16575307 0.41006614 0.90515072
 0.27753515 1.00271622 0.88587042 1.00751939 0.93572133 0.9566334
 1.07960267 0.63157343 1.03120912 0.98580148 1.03730711 0.6935431
 0.82780146 0.10370697 0.96685547 0.555259   0.94885848 1.02549746
 0.11812609 0.75309108 1.02551121 0.9092606  0.97438006 0.63486987
 0.87362238 0.09663744 0.76694669 1.01897093 0.96751052 1.03007861
 0.74120403 0.99582345 0.81639243 0.77507603 1.06239143 0.95847244
 0.95717761 0.70384014 0.96674135 0.65460556 0.01344722 0.48359847
 0.44449452 0.22270369 0.92998172 1.00539539 1.0223881  0.96475036
 0.59805812 0.40480652 0.07834212 0.44141082 0.20210165 0.61106316
 0.65061257 0.98892957 0.60102686]


In [385]:
win_above_seed = 0 
higher_seed_won = 0
prefer_higher_seed=0
missed = 0
for i in range(len(winner_names)):
    if winner_names[i] == actual_winners[i]: 
        if actual_seeds[i] > 8:
#             print("Win above seed!")
            win_above_seed += 1
        else:
#             print("Higher seed won")
            higher_seed_won += 1
    elif winner_names[i] != actual_winners[i]:
        if actual_seeds[i] > 8:
#             print("uh oh, model prefers higher seed")
            prefer_higher_seed +=1
        else:
#             print("Guessed: " + winner_names[i] + " Actual: " + actual_winners[i])
            missed += 1
    else: 
        print("what is this condition???")
        
print("Right: Win above seed!", win_above_seed)
print("Right: Higher seed won", higher_seed_won)
print("Wrong: uh oh, model prefers higher seed", prefer_higher_seed)
print("Wrong: missed but choose higher seed", missed)
print(len(winner_names))

Right: Win above seed! 0
Right: Higher seed won 0
Wrong: uh oh, model prefers higher seed 0
Wrong: missed but choose higher seed 0
0
